In [ ]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Maximalanzeigen für Pandas (vorsichtig nutzen bei großen Daten)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# --- Daten laden und vorbereiten ---
df = pd.read_csv("202408-bluebikes-tripdata.csv", sep=";", on_bad_lines="skip", low_memory=False)
df['started_at'] = pd.to_datetime(df['started_at'], dayfirst=True, errors='coerce')
df = df.dropna(subset=['started_at'])
df['date'] = df['started_at'].dt.normalize()
df['hour'] = df['started_at'].dt.hour.astype('int8')
df['weekday'] = df['started_at'].dt.dayofweek.astype('int8')

agg = df.groupby(['start_station_id', 'date', 'hour']).size().reset_index(name='count')
agg['weekday'] = pd.to_datetime(agg['date']).dt.dayofweek.astype('int8')

weather_data = {
    'date': pd.date_range("2024-08-01", "2024-08-31", freq='D').normalize(),
    'temp_high_F': [95, 88, 92, 88, 87, 78, 69, 71, 85, 89, 84, 79, 82, 79, 78, 72, 71, 69, 75, 70, 76, 76, 83, 84, 81, 77, 74, 91, 69, 70, 79],
    'temp_low_F':  [74, 72, 74, 70, 72, 62, 59, 60, 64, 74, 69, 65, 64, 66, 65, 64, 63, 65, 65, 60, 58, 58, 60, 62, 64, 65, 63, 65, 62, 58, 62],
    'precip_in':   [0.00,0.77,0.00,0.36,0.00,0.58,0.24,0.22,0.31,0.12,0.00,0.00,0.00,0.00,0.89,0.00,0.00,0.03,0.11,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]
}
weather_df = pd.DataFrame(weather_data)
agg = agg.merge(weather_df, on='date', how='left')

agg['start_station_id'] = agg['start_station_id'].astype('category')

# --- Features und Target ---
X = pd.DataFrame({
    'station_code': agg['start_station_id'].cat.codes,
    'hour': agg['hour'],
    'weekday': agg['weekday'],
    'temp_high_F': agg['temp_high_F'],
    'temp_low_F': agg['temp_low_F'],
    'precip_in': agg['precip_in']
})
y = agg['count']

# --- Modelltraining auf dem gesamten Datensatz ---
rf = RandomForestRegressor(n_jobs=-1, max_depth=10, random_state=42)
rf.fit(X, y)

# --- Vorhersagen für alle Daten ---
agg['prediction'] = rf.predict(X).round(1)

# --- Sortieren nach Datum ---
agg_sorted = agg.sort_values(by='date').reset_index(drop=True)

# Nur gewünschte Spalten
agg_display = agg_sorted[['date', 'start_station_id', 'prediction']]

# Ausgabe in Chunks (z.B. 5000 Zeilen pro Chunk)
chunk_size = 5000
for i in range(0, len(agg_display), chunk_size):
    display(HTML(agg_display.iloc[i:i+chunk_size].to_html(index=False)))

# Optional: kompletten Datensatz speichern
agg.to_csv("bike_predictions_all_data_with_full_predictions.csv", index=False)
print("✅ Datei 'bike_predictions_all_data_with_full_predictions.csv' gespeichert.")


In [ ]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# --- Funktion zum Einlesen und Aggregieren einzelner CSV-Dateien ---
def load_and_prepare(filepath):
    print(f"📂 Lade Datei: {filepath}")
    df = pd.read_csv(filepath, sep=";", on_bad_lines="skip", low_memory=False)

    # Spaltennamen vereinheitlichen
    df.columns = df.columns.str.strip().str.lower()

    if 'started_at' not in df.columns:
        raise ValueError(f"❌ Die Datei '{filepath}' enthält keine Spalte 'started_at'. Gefundene Spalten: {list(df.columns)}")

    df['started_at'] = pd.to_datetime(df['started_at'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['started_at'])
    df['date'] = df['started_at'].dt.normalize()
    df['hour'] = df['started_at'].dt.hour.astype('int8')
    df['weekday'] = df['started_at'].dt.dayofweek.astype('int8')
    agg = df.groupby(['start_station_id', 'date', 'hour']).size().reset_index(name='count')
    agg['weekday'] = pd.to_datetime(agg['date']).dt.dayofweek.astype('int8')
    return agg

# --- Alle drei Monatsdateien einlesen ---
agg_2022 = load_and_prepare("202208-bluebikes-tripdata.csv")
agg_2023 = load_and_prepare("202308-bluebikes-tripdata.csv")
agg_2024 = load_and_prepare("202408-bluebikes-tripdata.csv")

# --- Zusammenführen ---
agg_all = pd.concat([agg_2022, agg_2023, agg_2024], ignore_index=True)

# --- Wetterdaten für August 2024 ---
weather_data = {
    'date': pd.date_range("2024-08-01", "2024-08-31", freq='D').normalize(),
    'temp_high_F': [95, 88, 92, 88, 87, 78, 69, 71, 85, 89, 84, 79, 82, 79, 78, 72, 71, 69, 75, 70, 76, 76, 83, 84, 81, 77, 74, 91, 69, 70, 79],
    'temp_low_F':  [74, 72, 74, 70, 72, 62, 59, 60, 64, 74, 69, 65, 64, 66, 65, 64, 63, 65, 65, 60, 58, 58, 60, 62, 64, 65, 63, 65, 62, 58, 62],
    'precip_in':   [0.00,0.77,0.00,0.36,0.00,0.58,0.24,0.22,0.31,0.12,0.00,0.00,0.00,0.00,0.89,0.00,0.00,0.03,0.11,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]
}
weather_df = pd.DataFrame(weather_data)

# --- Wetterdaten zuordnen ---
agg_all = agg_all.merge(weather_df, on='date', how='left')
agg_all['temp_high_F'] = agg_all['temp_high_F'].fillna(weather_df['temp_high_F'].mean())
agg_all['temp_low_F'] = agg_all['temp_low_F'].fillna(weather_df['temp_low_F'].mean())
agg_all['precip_in'] = agg_all['precip_in'].fillna(0)

agg_all['start_station_id'] = agg_all['start_station_id'].astype('category')

# --- Features & Target ---
X = pd.DataFrame({
    'station_code': agg_all['start_station_id'].cat.codes,
    'hour': agg_all['hour'],
    'weekday': agg_all['weekday'],
    'temp_high_F': agg_all['temp_high_F'],
    'temp_low_F': agg_all['temp_low_F'],
    'precip_in': agg_all['precip_in']
})
y = agg_all['count']

# --- Modelltraining ---
rf = RandomForestRegressor(n_jobs=-1, max_depth=10, random_state=42)
rf.fit(X, y)

# --- Interaktive Vorhersagefunktion ---
def predict_count(date_str, hour, station_id):
    date = pd.to_datetime(date_str).normalize()
    weekday = date.dayofweek
    weather_row = weather_df[weather_df['date'] == date]

    if weather_row.empty:
        temp_high = weather_df['temp_high_F'].mean()
        temp_low = weather_df['temp_low_F'].mean()
        precip = 0
    else:
        temp_high = weather_row['temp_high_F'].values[0]
        temp_low = weather_row['temp_low_F'].values[0]
        precip = weather_row['precip_in'].values[0]

    if station_id not in agg_all['start_station_id'].unique():
        return f"❌ Station-ID {station_id} nicht im Datensatz enthalten."

    station_code = agg_all['start_station_id'].cat.categories.get_loc(station_id)

    input_features = pd.DataFrame([{
        'station_code': station_code,
        'hour': int(hour),
        'weekday': int(weekday),
        'temp_high_F': float(temp_high),
        'temp_low_F': float(temp_low),
        'precip_in': float(precip)
    }])

    prediction = rf.predict(input_features)[0]
    return f"📈 Vorhersage für {date_str} um {hour}:00 Uhr an Station {station_id}: {prediction:.1f} Fahrten"

# --- Beispielaufruf ---
print(predict_count("2024-08-17", 14, 31206.0))


In [ ]:
import pandas as pd
import os

# 🚀 Automatische Harmonisierung
def load_and_normalize(filepath):
    print(f"📥 Lade Datei: {filepath}")
    df = pd.read_csv(filepath, on_bad_lines='skip', low_memory=False)

    # 🔧 Spaltennamen vereinheitlichen
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

    # 🧭 Mögliche Umbenennungen ins Standardformat
    col_rename_map = {
        'starttime': 'started_at',
        'stoptime': 'ended_at',
        'start_station_id': 'start_station_id',
        'start_station_name': 'start_station_name',
        'end_station_id': 'end_station_id',
        'end_station_name': 'end_station_name',
        'usertype': 'member_casual',
        'bikeid': 'rideable_type'  # dummy fallback
    }

    # 🧠 Wenn 'started_at' nicht da ist, aber z. B. 'starttime', dann umbenennen
    for col_old, col_new in col_rename_map.items():
        if col_old in df.columns and col_new not in df.columns:
            df = df.rename(columns={col_old: col_new})

    # ✅ Vereinheitlichung: usertype → member_casual
    if 'member_casual' in df.columns:
        df['member_casual'] = df['member_casual'].replace({
            'Subscriber': 'member',
            'Customer': 'casual'
        })

    # 🕒 Zeitfelder zu datetime, falls vorhanden
    if 'started_at' in df.columns:
        df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
        df['date'] = df['started_at'].dt.date
        df['hour'] = df['started_at'].dt.hour
        df['weekday'] = df['started_at'].dt.dayofweek

    return df

# 🔄 Alle relevanten Dateien laden
files = ["202208-bluebikes-tripdata.csv", "202308-bluebikes-tripdata.csv", "202408-bluebikes-tripdata.csv"]

dfs = []
for file in files:
    if os.path.exists(file):
        df = load_and_normalize(file)
        dfs.append(df)
    else:
        print(f"⚠️ Datei nicht gefunden: {file}")

# 📊 Zusammenfügen
df_all = pd.concat(dfs, ignore_index=True)
print("✅ Alle Dateien wurden erfolgreich vereinheitlicht und zusammengeführt.")

# ✅ Beispiel für Weiterverarbeitung
# df_all.to_csv("bluebikes_all_normalized.csv", index=False)


In [ ]:
import pandas as pd
from datetime import datetime
from meteostat import Point, Daily

# --- Punkt für Boston (Koordinaten) ---
boston = Point(42.3601, -71.0589)

# --- Funktion zur Wetterabfrage für einen bestimmten August ---
def fetch_august_weather(year):
    start = datetime(year, 8, 1)   # <-- KEIN tzinfo mehr
    end   = datetime(year, 8, 31)

    data = Daily(boston, start, end)
    data = data.fetch().reset_index()

    # Umrechnungen: Celsius → Fahrenheit, mm → inch
    data['temp_high_F'] = data['tmax'] * 9 / 5 + 32
    data['temp_low_F']  = data['tmin'] * 9 / 5 + 32
    data['precip_in']   = data['prcp'] / 25.4

    # Relevante Spalten auswählen
    return data[['time', 'temp_high_F', 'temp_low_F', 'precip_in']].rename(columns={'time': 'date'})

# --- Wetterdaten für 2022 und 2023 abrufen ---
weather_2022 = fetch_august_weather(2022)
weather_2023 = fetch_august_weather(2023)

# --- Zusammenführen ---
weather_df = pd.concat([weather_2022, weather_2023], ignore_index=True)

# --- Ausgabe prüfen ---
print(weather_df)


In [ ]:
import pandas as pd

def load_and_prepare_bike_data(filenames):
    dfs = []

    for f in filenames:
        print(f"Lade Datei: {f}")
        df = pd.read_csv(f)

        # Versuche alle möglichen Namen für Zeitspalte
        time_col = None
        for candidate in ['started_at', 'start_time', 'starttime', 'start']:
            if candidate in df.columns:
                time_col = candidate
                break
        if not time_col:
            raise KeyError(f"Keine gültige Zeitspalte in Datei {f}")

        df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
        df = df.dropna(subset=[time_col])
        df['started_at'] = df[time_col]

        # Versuche alle möglichen Namen für Start-Station
        station_col = None
        for candidate in ['start_station_name', 'start station name', 'start_station', 'start location']:
            if candidate in df.columns:
                station_col = candidate
                break
        if not station_col:
            raise KeyError(f"Keine gültige Startstations-Spalte in Datei {f}")

        df = df.dropna(subset=[station_col])
        df['start_station_name'] = df[station_col]

        # Features erstellen
        df['date'] = df['started_at'].dt.date
        df['hour'] = df['started_at'].dt.hour

        # Aggregation
        df_agg = (
            df.groupby(['date', 'hour', 'start_station_name'])
            .size()
            .reset_index(name='trip_count')
        )
        dfs.append(df_agg)

    return pd.concat(dfs, ignore_index=True)


In [ ]:
import pandas as pd

# Liste der Datendateien (hier: August 2022, 2023, 2024)
files = ["202208-bluebikes-tripdata.csv", "202308-bluebikes-tripdata.csv", "202408-bluebikes-tripdata.csv"]

df_list = []
for file in files:
    df = pd.read_csv(file, sep=";", on_bad_lines="skip", low_memory=False)
    # Spalten bereinigen (evtl. Umbenennung, Kleinschreibung, Entfernen von Leerzeichen)
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")
    # Zeitsäule vereinheitlichen
    if 'started_at' in df.columns:
        time_col = 'started_at'
    elif 'starttime' in df.columns:
        time_col = 'starttime'
    else:
        raise KeyError(f"Keine gültige Zeit-Spalte in {file}")
    # Datum konvertieren (Tag zuerst, falls z.B. europ. Datumsformat)
    df[time_col] = pd.to_datetime(df[time_col], dayfirst=True, errors='coerce')
    df = df.dropna(subset=[time_col])  # ungültige Zeitstempel entfernen
    df['date'] = df[time_col].dt.date

    df_list.append(df)

# Alle Daten zusammenführen
df_all = pd.concat(df_list, ignore_index=True)


In [ ]:
# Anzahl der Fahrten pro Datum (Jahr-Monat-Tag) ermitteln
daily_counts = df_all.groupby('date').size().reset_index(name='rides')

# Datumsspalte als datetime-Objekt (optional)
daily_counts['date'] = pd.to_datetime(daily_counts['date'])


In [ ]:
from meteostat import Point, Daily
from datetime import datetime

# Wetterdaten für Boston (Koordinaten)
boston = Point(42.3601, -71.0589)

def fetch_august_weather(year):
    start = datetime(year, 8, 1)
    end   = datetime(year, 8, 31)
    data = Daily(boston, start, end)
    data = data.fetch().reset_index()
    # Konvertierungen: Celsius → Fahrenheit, mm → Zoll
    data['temp_high_F'] = data['tmax'] * 9/5 + 32
    data['temp_low_F']  = data['tmin'] * 9/5 + 32
    data['precip_in']   = data['prcp'] / 25.4
    return data[['time', 'temp_high_F', 'temp_low_F', 'precip_in']].rename(columns={'time': 'date'})

# Wetter für August 2022 und 2023 abrufen und zusammenführen
weather_2022 = fetch_august_weather(2022)
weather_2023 = fetch_august_weather(2023)
weather_df = pd.concat([weather_2022, weather_2023], ignore_index=True)


In [ ]:
# Datum in daily_counts bereits datetime, gleiche Basis wie weather_df
daily_counts = daily_counts.merge(weather_df, on='date', how='left')

# Fehlende Wetterwerte (z.B. für 2024 oder Tage ohne Daten) mit Mittelwerten auffüllen
daily_counts['temp_high_F'] = daily_counts['temp_high_F'].fillna(weather_df['temp_high_F'].mean())
daily_counts['temp_low_F']  = daily_counts['temp_low_F'].fillna(weather_df['temp_low_F'].mean())
daily_counts['precip_in']   = daily_counts['precip_in'].fillna(0)

# Weitere Merkmale extrahieren
daily_counts['year']    = daily_counts['date'].dt.year
daily_counts['month']   = daily_counts['date'].dt.month
daily_counts['weekday'] = daily_counts['date'].dt.dayofweek  # Montag=0, Sonntag=6


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Feature-Matrix und Zielwert
X = daily_counts[['year','month','weekday','temp_high_F','temp_low_F','precip_in']]
y = daily_counts['rides']

# Optional: Testdaten für Evaluierung abtrennen
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelltraining
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# Vorhersage auf Validierungsset
y_pred = rf.predict(X_val)
rise = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE auf Validierungsdaten: {rise:.1f} Fahrten")


In [ ]:
# Endgültiges Modell auf allen Daten trainieren
rf_final = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_final.fit(X, y)


In [ ]:
from datetime import datetime

def predict_rides(date_str):
    """
    Gibt die prognostizierte Anzahl der Fahrten für ein gegebenes Datum zurück.
    date_str: Datum als String im Format "YYYY-MM-DD"
    """
    # Datum verarbeiten
    date = datetime.strptime(date_str, "%Y-%m-%d")
    year = date.year
    month = date.month
    weekday = date.weekday()

    # Wetterdaten für das gegebene Datum abrufen (aus weather_df, das 2022+2023 enthält)
    date_pd = pd.to_datetime(date)
    row = weather_df[weather_df['date'] == date_pd]
    if not row.empty:
        temp_high = float(row['temp_high_F'])
        temp_low  = float(row['temp_low_F'])
        precip    = float(row['precip_in'])
    else:
        # Falls keine Wetterdaten vorhanden (z.B. künftiges Datum), mit Durchschnittswerten arbeiten
        temp_high = weather_df['temp_high_F'].mean()
        temp_low  = weather_df['temp_low_F'].mean()
        precip    = 0.0

    # Feature-Datenframe für die Vorhersage erstellen
    features = pd.DataFrame([{
        'year': year,
        'month': month,
        'weekday': weekday,
        'temp_high_F': temp_high,
        'temp_low_F': temp_low,
        'precip_in': precip
    }])

    # Vorhersage mit dem trainierten Random Forest
    prediction = rf_final.predict(features)[0]
    return prediction

# Beispiel: Vorhersage für den 1. August 2024 um 00:00 Uhr
date_input = "2024-08-5"
predicted_rides = predict_rides(date_input)
print(f"Vorhergesagte Anzahl der Fahrten am {date_input}: {predicted_rides:.0f}")


In [ ]:
import pandas as pd

def load_csv_safe(filepath, parse_dates=None):
    try:
        # Versuche mit Komma als Trennzeichen
        df = pd.read_csv(filepath, sep=';' , parse_dates=parse_dates)
        if 'started_at' not in df.columns or 'ended_at' not in df.columns:
            raise ValueError("Spalten fehlen – versuche Semikolon")
    except (ValueError, pd.errors.ParserError):
        # Fallback: Semikolon als Trennzeichen
        df = pd.read_csv(filepath, delimiter=';', parse_dates=parse_dates)
    return df

def load_2024_data(filepath):
    df = load_csv_safe(filepath,  parse_dates=['started_at', 'ended_at'])
    df_clean = df[['ride_id', 'rideable_type', 'started_at', 'ended_at',
                   'start_station_name', 'start_station_id',
                   'end_station_name', 'end_station_id',
                   'start_lat', 'start_lng', 'end_lat', 'end_lng',
                   'member_casual']].copy()
    df_clean.rename(columns={'member_casual': 'user_type'}, inplace=True)
    return df_clean

def load_2023_data(filepath):
    df1 = load_csv_safe(filepath, parse_dates=['started_at', 'ended_at'])
    df_clean = df1[['ride_id', 'rideable_type', 'started_at', 'ended_at',
                   'start_station_name', 'start_station_id',
                   'end_station_name', 'end_station_id',
                   'start_lat', 'start_lng', 'end_lat', 'end_lng',
                   'member_casual']].copy()
    df_clean.rename(columns={'member_casual': 'user_type'}, inplace=True)
    return df_clean

def load_2022_data(filepath):
    df2 = load_csv_safe(filepath, parse_dates=['started_at', 'ended_at'])
    df_clean = df2[['tripduration', 'started_at', 'ended_at',
                   'start station name', 'start station id',
                   'end station name', 'end station id',
                   'start station latitude', 'start station longitude',
                   'end station latitude', 'end station longitude',
                   'bikeid', 'usertype']].copy()
    df_clean.rename(columns={
        'start station name': 'start_station_name',
        'start station id': 'start_station_id',
        'end station name': 'end_station_name',
        'end station id': 'end_station_id',
        'start station latitude': 'start_lat',
        'start station longitude': 'start_lng',
        'end station latitude': 'end_lat',
        'end station longitude': 'end_lng',
        'usertype': 'user_type'
    }, inplace=True)
    return df_clean

# --- DATEN LADEN ---

df2024 = load_2024_data("202408-bluebikes-tripdata.csv")
df2023 = load_2023_data("202308-bluebikes-tripdata.csv")
df2022 = load_2022_data("202208-bluebikes-tripdata.csv")

# Gemeinsame Spalten erzwingen
common_cols = list(set(df2024.columns) & set(df2023.columns) & set(df2022.columns))
df_all = pd.concat([df2024[common_cols], df2023[common_cols], df2022[common_cols]], ignore_index=True)

print("Daten erfolgreich geladen.")
print(df_all.head())


In [ ]:
import pandas as pd

# ---------------------
# 1. CSV-Dateien laden
# ---------------------

df2022 = pd.read_csv('202208-bluebikes-tripdata.csv', sep=';', parse_dates=['started_at', 'ended_at'])
df2023 = pd.read_csv('202308-bluebikes-tripdata.csv', sep=';', parse_dates=['started_at', 'ended_at'])
df2024 = pd.read_csv('202408-bluebikes-tripdata.csv', sep=';', parse_dates=['started_at', 'ended_at'])

# ----------------------------
# 2. Spalten vereinheitlichen
# ----------------------------

# 2022 hat andere Spaltennamen → umbenennen
df2022 = df2022.rename(columns={
    'start station id': 'start_station_id',
    'start station name': 'start_station_name',
    'end station id': 'end_station_id',
    'end station name': 'end_station_name',
    'start station latitude': 'start_lat',
    'start station longitude': 'start_lng',
    'end station latitude': 'end_lat',
    'end station longitude': 'end_lng',
    'tripduration': 'trip_duration',
    'bikeid': 'ride_id',
    'usertype': 'member_casual'
})

# 2023 und 2024 haben dieselbe Struktur → kein Rename nötig

# Sicherstellen, dass alle DataFrames dieselben Spalten haben (ggf. unnötige Spalten entfernen)


# -------------------------------
# 3. Alle Jahre zu einem Dataset
# -------------------------------

data = pd.concat([df2022, df2023, df2024], ignore_index=True)





In [ ]:
from datetime import datetime
from meteostat import Point, Daily

# Standort Köln (Latitude, Longitude, Höhenmeter, optional)
koeln = Point(50.9375, 6.9603, 100)

# Zeitraum: Vom frühesten bis spätesten Datum aus den CSV-Daten
start = data['date'].min()
end   = data['date'].max()

# Abrufen der täglichen Wetterdaten (2020 als Beispieljahr)
weather = Daily(koeln, start, end).fetch()

# Auswahl typischer Wettermerkmale (tavg=°C, prcp=mm, wspd=Windgeschw.)
data = data.merge(weather[['tavg', 'prcp', 'wspd']], on='date', how='left')


In [ ]:
# Installation der notwendigen Pakete (ausführbar in einer Jupyter-Umgebung)
# Falls Du lokal arbeitest, bitte diese Zeile in Deiner Shell ausführen:
# pip install pandas numpy scikit-learn meteostat

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from meteostat import Daily, Stations
from datetime import datetime
import matplotlib.pyplot as plt

# Schritt 1: CSV-Dateien einlesen (Anpassung der Spaltennamen je nach Jahr)
df_2022 = pd.read_csv("202208-bluebikes-tripdata.csv",  sep=';' ,parse_dates=['started_at', 'ended_at'])
df_2023 = pd.read_csv("202308-bluebikes-tripdata.csv",  sep=';' ,parse_dates=['started_at', 'ended_at'])
df_2024 = pd.read_csv("202408-bluebikes-tripdata.csv",  sep=';' ,parse_dates=['started_at', 'ended_at'])

# Spalten vereinheitlichen
df_2022.rename(columns={
    'start station id': 'start_station_id',
    'start station name': 'start_station_name',
    'start station latitude': 'start_lat',
    'start station longitude': 'start_lng',
    'end station id': 'end_station_id',
    'end station name': 'end_station_name',
    'end station latitude': 'end_lat',
    'end station longitude': 'end_lng'
}, inplace=True)
for df in [df_2022, df_2023, df_2024]:
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df.dropna(subset=['started_at', 'ended_at'], inplace=True)
    df = df[df['ended_at'] > df['started_at']]
    df['duration_min'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60



# Einheitliche Dauerberechnung
df_2022['duration_min'] = (df_2022['ended_at'] - df_2022['started_at']).dt.total_seconds() / 60
df_2023['duration_min'] = (df_2023['ended_at'] - df_2023['started_at']).dt.total_seconds() / 60
df_2024['duration_min'] = (df_2024['ended_at'] - df_2024['started_at']).dt.total_seconds() / 60



# Alle zusammenführen
df_all = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

# Neue Spalten
df_all['date'] = df_all['started_at'].dt.date
df_all['hour'] = df_all['started_at'].dt.hour
df_all['weekday'] = df_all['started_at'].dt.weekday

# Gruppierung
agg = df_all.groupby(['start_station_id', 'date', 'hour']).size().reset_index(name='count')

# Top 20 Stationen
top_stations = agg.groupby('start_station_id')['count'].sum().sort_values(ascending=False).head(20).index
agg = agg[agg['start_station_id'].isin(top_stations)]

# Schritt 2: Wetterdaten Boston Logan Airport (72509)
start = datetime(2022, 8, 1)
end = datetime(2024, 8, 31)
weather = Daily('72509', start, end)
weather = weather.fetch().reset_index()
weather['date'] = weather['time'].dt.date

# Umrechnung °C -> °F und mm -> inch
weather['tmin_F'] = weather['tmin'] * 9/5 + 32
weather['tmax_F'] = weather['tmax'] * 9/5 + 32
weather['prcp_in'] = weather['prcp'] / 25.4

weather = weather[['date', 'tmin_F', 'tmax_F', 'prcp_in']]

# Merge
df_final = pd.merge(agg, weather, on='date', how='left')

# Hinzufügen von weekday
df_final['weekday'] = pd.to_datetime(df_final['date']).dt.weekday

# Modellierung
X = df_final[['start_station_id', 'hour', 'weekday', 'tmax_F', 'tmin_F', 'prcp_in']]
y = df_final['count']

# One-Hot-Encoding für start_station_id
X = pd.get_dummies(X, columns=['start_station_id'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluation
from sklearn.metrics import mean_squared_error
import numpy as np



# In NumPy-Arrays umwandeln und flach machen
y_test = np.array(y_test).astype(float).ravel()
y_pred = np.array(y_pred).astype(float).ravel()

# Prüfen auf NaN oder inf
if np.isnan(y_test).any() or np.isnan(y_pred).any():
    raise ValueError("y_test oder y_pred enthält NaN-Werte")
if np.isinf(y_test).any() or np.isinf(y_pred).any():
    raise ValueError("y_test oder y_pred enthält Inf-Werte")
print("Typ y_test:", type(y_test))
print("Typ y_pred:", type(y_pred))
print("Shape y_test:", np.shape(y_test))
print("Shape y_pred:", np.shape(y_pred))
print("Erste Werte y_test:", y_test[:5])
print("Erste Werte y_pred:", y_pred[:5])

import numpy as np

def rmse_manual(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    return np.sqrt(mse)

rmse = rmse_manual(y_test, y_pred)
print("Manueller RMSE:", rmse)



In [ ]:
# Installation der notwendigen Pakete (ausführbar in einer Jupyter-Umgebung)
# Falls Du lokal arbeitest, bitte diese Zeile in Deiner Shell ausführen:
# pip install pandas numpy scikit-learn meteostat

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from meteostat import Daily, Stations
from datetime import datetime
import matplotlib.pyplot as plt

# Schritt 1: CSV-Dateien einlesen (Anpassung der Spaltennamen je nach Jahr)
df_2022 = pd.read_csv("202208-bluebikes-tripdata.csv",  sep=';' ,parse_dates=['started_at', 'ended_at'])
df_2023 = pd.read_csv("202308-bluebikes-tripdata.csv",  sep=';' ,parse_dates=['started_at', 'ended_at'])
df_2024 = pd.read_csv("202408-bluebikes-tripdata.csv",  sep=';' ,parse_dates=['started_at', 'ended_at'])

# Spalten vereinheitlichen
df_2022.rename(columns={
    'start station id': 'start_station_id',
    'start station name': 'start_station_name',
    'start station latitude': 'start_lat',
    'start station longitude': 'start_lng',
    'end station id': 'end_station_id',
    'end station name': 'end_station_name',
    'end station latitude': 'end_lat',
    'end station longitude': 'end_lng'
}, inplace=True)
for df in [df_2022, df_2023, df_2024]:
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df.dropna(subset=['started_at', 'ended_at'], inplace=True)
    df = df[df['ended_at'] > df['started_at']]
    df['duration_min'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60



# Einheitliche Dauerberechnung
df_2022['duration_min'] = (df_2022['ended_at'] - df_2022['started_at']).dt.total_seconds() / 60
df_2023['duration_min'] = (df_2023['ended_at'] - df_2023['started_at']).dt.total_seconds() / 60
df_2024['duration_min'] = (df_2024['ended_at'] - df_2024['started_at']).dt.total_seconds() / 60



# Alle zusammenführen
df_all = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

# Neue Spalten
df_all['date'] = df_all['started_at'].dt.date
df_all['hour'] = df_all['started_at'].dt.hour
df_all['weekday'] = df_all['started_at'].dt.weekday

# Gruppierung
agg = df_all.groupby(['start_station_id', 'date', 'hour']).size().reset_index(name='count')

# Top 20 Stationen
top_stations = agg.groupby('start_station_id')['count'].sum().sort_values(ascending=False).head(20).index
agg = agg[agg['start_station_id'].isin(top_stations)]

# Schritt 2: Wetterdaten Boston Logan Airport (72509)
start = datetime(2022, 8, 1)
end = datetime(2024, 8, 31)
weather = Daily('72509', start, end)
weather = weather.fetch().reset_index()
weather['date'] = weather['time'].dt.date

# Umrechnung °C -> °F und mm -> inch
weather['tmin_F'] = weather['tmin'] * 9/5 + 32
weather['tmax_F'] = weather['tmax'] * 9/5 + 32
weather['prcp_in'] = weather['prcp'] / 25.4

weather = weather[['date', 'tmin_F', 'tmax_F', 'prcp_in']]

# Merge
df_final = pd.merge(agg, weather, on='date', how='left')

# Hinzufügen von weekday
df_final['weekday'] = pd.to_datetime(df_final['date']).dt.weekday

# --- 3. Feature Engineering ---
X = df_final[['start_station_id', 'hour', 'weekday', 'tmax_F', 'tmin_F', 'prcp_in']]
y = df_final['count']

# One-Hot-Encoding der Stationen
X = pd.get_dummies(X, columns=['start_station_id'])

# Trainings- und Testsplit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelltraining
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# --- 4. Vorhersage-Funktion ---
def predict_rides(model, station_id, date_str, hour, weather_df, feature_cols):
    """
    Prognose der Anzahl der Fahrten für eine bestimmte Station, Datum und Stunde.

    Args:
        model: Trainiertes RandomForestRegressor-Modell.
        station_id (int oder str): Startstation-ID.
        date_str (str): Datum im Format 'YYYY-MM-DD'.
        hour (int): Stunde des Tages (0-23).
        weather_df (pd.DataFrame): Wetterdaten mit Spalten ['date', 'tmax_F', 'tmin_F', 'prcp_in'].
        feature_cols (pd.Index): Feature-Spalten des trainierten Modells (One-Hot-Kodierung).

    Returns:
        float: Vorhergesagte Anzahl der Fahrten.
    """
    date = pd.to_datetime(date_str).date()
    weekday = pd.to_datetime(date_str).weekday()

    # Wetterdaten für das Datum holen
    weather_row = weather_df[weather_df['date'] == date]
    if weather_row.empty:
        raise ValueError(f"Wetterdaten für {date} nicht gefunden.")

    tmax_F = weather_row['tmax_F'].values[0]
    tmin_F = weather_row['tmin_F'].values[0]
    prcp_in = weather_row['prcp_in'].values[0]

    # Feature-Vektor initialisieren
    data = {
        'hour': hour,
        'weekday': weekday,
        'tmax_F': tmax_F,
        'tmin_F': tmin_F,
        'prcp_in': prcp_in
    }

    # One-Hot-Encoding der Stationen übernehmen
    for col in feature_cols:
        if col.startswith('start_station_id_'):
            data[col] = 1 if col == f'start_station_id_{station_id}' else 0

    # DataFrame für Modellinput
    X_pred = pd.DataFrame([data], columns=feature_cols)

    # Vorhersage
    prediction = model.predict(X_pred)[0]
    return prediction

# --- 5. Beispielvorhersage ---
feature_cols = X.columns
example_station = top_stations[0]  # z.B. erste Top-Station
example_date = '2024-08-15'
example_hour = 14

predicted_count = predict_rides(model, example_station, example_date, example_hour, weather, feature_cols)
print(f"Vorhergesagte Anzahl Fahrten am {example_date} um {example_hour} Uhr an Station {example_station}: {predicted_count:.2f}")
